<h1 align="center"> Tutorial: scikit-learn new features</h1>

<br />
<div align="center">April 28, 2019</div>
<br />

<div align="center">Roman Yurchak (notebook adapted from work by Olivier Grisel)</div>

Running this notebooks requires Python 3.5+ as well as,
 - scikit-learn >=0.21.2
 - matplotlib
 - pandas

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

## 1. Example: the California housing dataset

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import median_absolute_error


calhousing = fetch_california_housing()

X = pd.DataFrame(calhousing.data, columns=calhousing.feature_names)
y = pd.Series(calhousing.target, name='house_value')

In [ ]:
print(calhousing.DESCR)

In [ ]:
X.head()

In [ ]:
y.head().to_frame()

## 2. Spatial clustering

Let's looks at the spatial distribution of census block groups, and the corresponding median house value,

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

X.plot.scatter('Longitude', 'Latitude', c=y, ax=ax,
               colorbar=True, colormap='Reds', alpha=0.7, s=10)
ax.set_title(y.name);

Could we try to detect geographical entities (towns, cities) from this spatial distribution?

In [ ]:
%%time

from sklearn.cluster import DBSCAN


dbscan = DBSCAN(eps=0.1).fit(X[['Longitude', 'Latitude']])


fig, ax = plt.subplots(1, 1, figsize=(8, 5))

labels = dbscan.labels_
labels[labels>=0] += 5
print('n_clusters:', len(np.unique(labels)))

X.plot.scatter('Longitude', 'Latitude', c=labels, ax=ax,
               colorbar=True, colormap='viridis', alpha=0.7, s=10);

Samples that don't belong to any cluster (noisy samples) have a value of -1.

The choice of `eps` parameter will strongly impact both the number of clusters and the fraction of noisy samples in DBSCAN.

In [ ]:
%%time

from sklearn.cluster import OPTICS

cl = OPTICS().fit(X[['Longitude', 'Latitude']])

fig, ax = plt.subplots(1, 1, figsize=(8, 5))

labels = cl.labels_
labels[labels>=0] += 5
print('n_clusters:', len(np.unique(labels)))

X.plot.scatter('Longitude', 'Latitude', c=labels, ax=ax,
               colorbar=True, colormap='viridis', alpha=0.7, s=10);

## 3. Baseline supervised model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1000, random_state=0)

Let's start with a quick baseline model: linear regression (aka. Ordinary Least Squares):

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

%time lm = LinearRegression().fit(X_train, y_train)

In [ ]:
print("train error: %0.3f, test error: %0.3f" %
      (median_absolute_error(y_train, lm.predict(X_train)),
       median_absolute_error(y_test, lm.predict(X_test))))

In [ ]:
def scatter_predictions(y_pred, y_true):
    plt.figure(figsize=(6, 6))
    plt.xlabel('prediction')
    plt.ylabel('true target')
    plt.xlim(-1, 6)
    plt.ylim(-1, 6)
    plt.scatter(y_pred, y_true)
    
scatter_predictions(lm.predict(X_test), y_test)

This is pretty bad: the errors (off-diagonal predictions) seems to be heteroschedastic and there is a saturation effect with many samples with `y_true == 5`. Let's check:

In [ ]:
ax = y.hist(bins=50)

ax.set_xlabel(y.name);

We can filter out the "anomalies" and make the target variable marginal distribution more "Gaussian" by taking the log:

In [ ]:
np.log(y[y<4.9]).hist(bins=50);

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X[y<4.9], y[y<4.9], test_size=1000, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


lm2 = make_pipeline(StandardScaler(), LinearRegression())
_ = %time lm2.fit(X_train, np.log(y_train))

In [ ]:
print("train error: %0.3f, test error: %0.3f" %
      (median_absolute_error(y_train,
                             np.exp(lm2.predict(X_train))),
       median_absolute_error(y_test,
                             np.exp(lm2.predict(X_test)))))

In [ ]:
scatter_predictions(
    np.exp(lm2.predict(X_test)),
    y_test)

## 4. Feature engineering / preprocessing

To facilitate the evaluation of subsequent models, let's factorize the evaluation code into a separate function,

In [ ]:
from pickle import dumps

def train_score_model(
        model, X_train, X_test, y_train, y_test, plot=False
):
    %time model.fit(X_train, np.log(y_train))

    print("train error: %0.3f, test error: %0.3f" %
          (median_absolute_error(
              y_train, np.exp(model.predict(X_train))),
           median_absolute_error(
              y_test,  np.exp(model.predict(X_test)))))
    
    model_size = len(dumps(model)) / 1e6
    if model_size > 0.1:
        print("Model size: %0.2f MB" % (model_size))

    if plot:
        scatter_predictions(
            np.exp(model.predict(X_test)), y_test
        )
    return model

train_score_model(lm2, X_train, X_test, y_train, y_test);

Now, let's examine more closely the distribution of different features. Most are assymetric distributions with a long tail.

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(12, 8))
X.hist(ax=ax, bins=50);
ax[0,0].set_xlim(0, 10)
ax[0,1].set_xlim(0, 200)
ax[0,2].set_xlim(0, 50);

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import make_union

plm = make_pipeline(
    ColumnTransformer([
        ('scaler', StandardScaler(),
         ['Latitude', 'Longitude', "HouseAge"]),
        ('power_transform', PowerTransformer(),
         ['MedInc', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup'])
    ]),
    LinearRegression())

train_score_model(plm, X_train, X_test, y_train, y_test);

We use `ColumnsTransformer` to selectively apply Yeo-Johnson power transform on some features, while keeping `StandardScaler` for others.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_union

plm = make_pipeline(
    StandardScaler(),
    make_union(
        FunctionTransformer(validate=True),
        PolynomialFeatures(degree=3)
    ),
    LinearRegression())

train_score_model(plm, X_train, X_test, y_train, y_test);

## 5. Model improvement

In this part, we keep the baseline feature scaling, while evaluating more advanced models.

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = make_pipeline(
    StandardScaler(),
    MLPRegressor(hidden_layer_sizes=(100, 10, 10), activation='relu'),
)

train_score_model(mlp, X_train, X_test, y_train, y_test);

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(learning_rate=0.1, max_depth=8, min_samples_leaf=20,
                                n_estimators=100, loss='huber')

train_score_model(gbr, X_train, X_test, y_train, y_test, plot=True);

In [ ]:
%timeit gbr.predict(X_test[:100])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)

train_score_model(rfr, X_train, X_test, y_train, y_test);

In [ ]:
%timeit rfr.predict(X_test[:100])

#### New Histogram-based Gradient Boosting Trees

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
hgbr = HistGradientBoostingRegressor(min_samples_leaf=20, max_leaf_nodes=256,
                                     max_iter=100)
train_score_model(hgbr, X_train, X_test, y_train, y_test);

In [ ]:
%timeit hgbr.predict(X_test[:100])

## 6. Early stopping

In [ ]:
hgbr = HistGradientBoostingRegressor(
    min_samples_leaf=20, max_leaf_nodes=256,
    n_iter_no_change=5, validation_fraction=0.1,
    scoring="loss", max_iter=10000
)

train_score_model(hgbr, X_train, X_test, y_train, y_test);

In [ ]:
hgbr.n_iter_

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(hgbr.train_score_, label="train")
ax.plot(hgbr.validation_score_, "--", label="validation")
ax.set_xlabel("Boosting iteration (trees)")
ax.set_ylabel("Negative loss (MSE)")
ax.legend();

## 7. Imputation

Let's artificially create some missing data so that we can illustrate the imputation estimators.

In [ ]:
rng = np.random.RandomState(42)

density = 4  # one in 10 values will be NaN

mask = rng.randint(density, size=X.shape) == 0
X_na = X.copy()
X_na.values[mask] = np.nan
X_na.head()

In [ ]:
X_train_na, X_test_na, y_train_na, y_test_na = train_test_split(
    X_na[y<4.9], y[y<4.9], test_size=1000, random_state=0)

In [ ]:
from sklearn.impute import SimpleImputer

model = make_pipeline(
    StandardScaler(),
    SimpleImputer(strategy="median"),
    LinearRegression()
)
print('Baseline model\n')
train_score_model(model, X_train, X_test, y_train, y_test);
print('\nBaseline model with missing data + SimpleImputer\n')
train_score_model(model, X_train_na, X_test_na, y_train_na, y_test_na);

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

pipe = make_pipeline(
    StandardScaler(),
    IterativeImputer(),
    LinearRegression()
)

train_score_model(pipe, X_train_na, X_test_na, y_train_na, y_test_na);

With `IterativeImputer` we almost fully compensate the existence of missing values with respect to the baseline performance.

## Tricks

### Warm started models

In [ ]:
X_train_small, X_val, y_train_small, y_val = train_test_split(
    X_train, y_train, train_size=5000, test_size=1000)

val_errors = []
train_errors = []


gbr = GradientBoostingRegressor(learning_rate=0.1, max_depth=8,
                                min_samples_leaf=3, n_estimators=1)

gbr.fit(X_train_small, y_train_small)

train_error = median_absolute_error(y_train, gbr.predict(X_train))
val_error = median_absolute_error(y_test, gbr.predict(X_test))

train_errors.append(train_error)
val_errors.append(val_error)

print("train error: %0.3f, test error: %0.3f" % (train_error, val_error)) 

In [ ]:
for i in range(100):
    gbr.set_params(warm_start=True, n_estimators=len(gbr.estimators_) + 1)
    gbr.fit(X_train_small, y_train_small)
    train_error = median_absolute_error(y_train, gbr.predict(X_train))
    val_error = median_absolute_error(y_test, gbr.predict(X_test))

    train_errors.append(train_error)
    val_errors.append(val_error)
    if (i + 2) % 10 == 0:
        print("n_trees=%d, train error: %0.3f, test error: %0.3f"
              % (len(gbr.estimators_), train_error, val_error)) 

In [ ]:
tree_indices = np.arange(len(val_errors)) + 1
plt.plot(tree_indices, train_errors, label='training')
plt.plot(tree_indices, val_errors, label='validation')
plt.xlabel('number of trees')
plt.ylabel('mean absolute error')
plt.legend(loc='best');

### Categorical features encoded as integers

One-hot encoding is pretty useless for tree-based models (at least in scikit-learn). Contrary to other models it's pretty safe and much more efficient to use integer based encoding for instance using pandas:

```python
    >>> categorical_data.apply(lambda x: pd.factorize(x)[0])
```

alternatively you can use scikit-learn's [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).